In [1]:
import jupyros as jr
import rospy
import tf
import ipywidgets as widgets
import math

from geometry_msgs.msg import PoseStamped
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry

from robotcontroller import RobotController
from driver import Driver

rospy.init_node('jupyter_cmdvel_node')

In [2]:
cmdvelpub = rospy.Publisher('cmd_vel', Twist, queue_size=10)

mapframe = 'odom'
transformlistener = tf.TransformListener()

robotcontroller = RobotController(transformlistener, mapframe, Driver(cmdvelpub))
jr.subscribe('/odom', Odometry, robotcontroller.newodometry)

In [3]:
leftbutton = widgets.Button(description="Left", layout=widgets.Layout(width='auto', grid_area='left'), style=widgets.ButtonStyle())
rightbutton = widgets.Button(description="Right", layout=widgets.Layout(width='auto', grid_area='right'), style=widgets.ButtonStyle())
stopbutton = widgets.Button(description="Stop", layout=widgets.Layout(width='auto', grid_area='center'), style=widgets.ButtonStyle())
forwardbutton = widgets.Button(description="Forward", layout=widgets.Layout(width='auto', grid_area='top'), style=widgets.ButtonStyle())
backwardbutton = widgets.Button(description="Backward", layout=widgets.Layout(width='auto', grid_area='bottom'), style=widgets.ButtonStyle())

def turn_left(b):
    tw = Twist()
    tw.linear.x = .0
    tw.linear.y = .0
    tw.linear.z = .0
    tw.angular.x = .0
    tw.angular.y = .0
    tw.angular.z = 1.2
    cmdvelpub.publish(tw)
    pass

def turn_right(b):
    tw = Twist()
    tw.linear.x = .0
    tw.linear.y = .0
    tw.linear.z = .0
    tw.angular.x = .0
    tw.angular.y = .0
    tw.angular.z = -1.2
    cmdvelpub.publish(tw)
    pass

def forward(b):
    tw = Twist()
    tw.linear.x = 0.3
    tw.linear.y = .0
    tw.linear.z = .0
    tw.angular.x = .0
    tw.angular.y = .0
    tw.angular.z = .0
    cmdvelpub.publish(tw)
    pass

def backward(b):
    tw = Twist()
    tw.linear.x = -0.3
    tw.linear.y = .0
    tw.linear.z = .0
    tw.angular.x = .0
    tw.angular.y = .0
    tw.angular.z = .0
    cmdvelpub.publish(tw)
    pass

def stop(b):
    tw = Twist()    
    tw.linear.x = .0
    tw.linear.y = .0
    tw.linear.z = .0
    tw.angular.x = .0
    tw.angular.y = .0
    tw.angular.z = .0
    cmdvelpub.publish(tw)
    
    robotcontroller.stop()
    pass

leftbutton.on_click(turn_left)
rightbutton.on_click(turn_right)
forwardbutton.on_click(forward)
backwardbutton.on_click(backward)
stopbutton.on_click(stop)

grid = widgets.GridspecLayout(3, 3)
grid[0, 1] = forwardbutton
grid[1,0] = leftbutton
grid[1,1] = stopbutton
grid[1,2] = rightbutton
grid[2,1] = backwardbutton

display(grid)

GridspecLayout(children=(Button(description='Forward', layout=Layout(grid_area='widget001', width='auto'), sty…

In [4]:
from robotbehaviors import RotateToState, DriveToPosition
from nav_msgs.msg import Path
from map import Map
import traceback

mapframe = 'odom'

map = Map()

pathpub = rospy.Publisher('navpath', Path, queue_size=10)

def navgoal(message):

    if robotcontroller.latestodominmapframe:

        navgoalinframe = transformlistener.transformPose(mapframe, message)

        #robotx = robotcontroller.latestodominmapframe.pose.position.x
        #roboty = robotcontroller.latestodominmapframe.pose.position.y

        #dx = navgoalinframe.pose.position.x - robotx
        #dy = navgoalinframe.pose.position.y - roboty

        #yaw = math.atan2(dy, dx)

        try:
            rospy.loginfo('Planning path...')
            
            p = map.findpath(robotcontroller.latestodominmapframe, navgoalinframe)
            if p is not None:
                path = map.gridpointstoposes(p, 'map')

                rospy.loginfo('Path calculated with %s poses', len(path.poses))
                pathpub.publish(path)
                             
                x = len(p) - 1
                while x >= 1:
                    (xtarget, ytarget) = p[x]
                    (xorigin, yorigin) = p[x - 1]
                    
                    pose = map.gridtopose(p[x])
                    path.poses.append(pose)

                    robotcontroller.appendbehavior(DriveToPosition(pose))

                    dx = xtarget - xorigin
                    dy = ytarget - yorigin

                    yaw = math.atan2(dy, dx)
                    robotcontroller.appendbehavior(RotateToState(yaw))
                    
                    x = x -1
                    
                # Calculate to first waypoint
                (xtarget, ytarget) = p[1]
                (xorigin, yorigin) = p[0]
                
                pose = map.gridtopose(p[1])
                path.poses.append(pose)

                robotcontroller.appendbehavior(DriveToPosition(pose))

                dx = xtarget - xorigin
                dy = ytarget - yorigin

                yaw = math.atan2(dy, dx)
                robotcontroller.appendbehavior(RotateToState(yaw))

                rospy.loginfo('Navigation state chain computed')

            else:
                rospy.loginfo('No path found')
                
        except Exception as e:
            rospy.loginfo('Error creating path : %s', traceback.format_exc())

    return

jr.subscribe('/move_base_simple/goal', PoseStamped, navgoal)

In [5]:
from nav_msgs.msg import OccupancyGrid
from ipycanvas import Canvas, hold_canvas

canvas = Canvas(width=300, height=500)
display(canvas)

coveragepathpub = rospy.Publisher('coveragepath', Path, queue_size=10)

def newnavgrid(message):

    try:
        map.newmap(message)

        with hold_canvas():
            canvas.clear()
            canvas.layout.width = str(message.info.width * 2) + 'px'
            canvas.layout.height = str(message.info.height * 2) + 'px'
            canvas.width = message.info.width
            canvas.height = message.info.height

            for y in range(message.info.height):
                for x in range(message.info.width):
                    bottomx = x;
                    bottomy = message.info.height - 1 - y

                    value = message.data[y * message.info.width + x]

                    if value == 0:
                        # Free cell
                        canvas.fill_style = "lightgray"
                        canvas.stroke_style = "lightgray"

                        canvas.fill_rect(bottomx, bottomy - 1, 1, 1)
                    if value > 0:
                        # Occupied
                        x = str(value)
                        canvas.fill_style = "rgba(0,0,255," + str(255 / value) + ")"
                        canvas.stroke_style = "rgba(0,0,255," + str(255 / value) + ")"

                        canvas.fill_rect(bottomx, bottomy - 1, 1, 1)
                            
            if robotcontroller.latestodominmapframe:
                robotx = robotcontroller.latestodominmapframe.pose.position.x
                roboty = robotcontroller.latestodominmapframe.pose.position.y

                mapx = int((robotx - message.info.origin.position.x) / message.info.resolution)
                mapy = int((roboty - message.info.origin.position.y) / message.info.resolution)

                canvas.fill_style = "red"
                canvas.stroke_style = "red"

                bottomy = message.info.height - 1 - mapy
                canvas.fill_rect(mapx, bottomy + 1, 4, 4)

            slices = map.celldecompose(5, None)
            for slice in slices:
                topx, ys = slice.top()
                bottomx, yd = slice.bottom()
                
                ys = message.info.height - slice.ystart
                yd = message.info.height - slice.yend
                
                canvas.fill_style = "yellow"
                canvas.stroke_style = "yellow"
                canvas.stroke_line(topx, ys, bottomx, yd)
                    
            if robotcontroller.latestodominmapframe:
                path, lastpos, error = map.fullcoveragepath(3, robotcontroller.latestodominmapframe, None)
                
                if path is not None:
                    cp = Path()
                    cp.header.frame_id = 'map'
                    cp.header.stamp = rospy.Time.now()

                    for idx, element in enumerate(path):
                        (ex, ey) = element                    
                        if idx > 0:
                            (sx, sy) = path[idx - 1]

                            sym = message.info.height - sy
                            eym = message.info.height - ey

                            canvas.fill_style = "green"
                            canvas.stroke_style = "green"
                            canvas.stroke_line(sx, sym, ex, eym)

                        cp.poses.append(map.gridtopose(element))

                    coveragepathpub.publish(cp)

                    if error:
                        (sx, sy) = lastpos
                        (ex, ey) = error

                        sym = message.info.height - sy
                        eym = message.info.height - ey

                        canvas.fill_style = "rgba(255,0,0,0.5)"
                        canvas.stroke_style = "rgba(255,0,0,0.5)"
                        canvas.stroke_line(sx, sym, ex, eym)

                        rospy.loginfo("Cannot create path from %s to %s : Direct path %s", str(lastpos), str(error), map.isdirectpath(lastpos, error))

    except Exception as e:
        rospy.loginfo('Error creating path : %s', traceback.format_exc())


jr.subscribe('/costmap_node/costmap/costmap', OccupancyGrid, newnavgrid)

Canvas(width=300)

         at line 278 in /opt/conda/build_artifacts/ros-noetic-tf2_1615118253106/work/ros-noetic-tf2/src/work/src/buffer_core.cpp
         at line 278 in /opt/conda/build_artifacts/ros-noetic-tf2_1615118253106/work/ros-noetic-tf2/src/work/src/buffer_core.cpp
